In [2]:
from Yahoo_TW import *
import numpy as np
import yfinance as yf
import pandas as pd
import quantstats as qs
import datetime
import warnings
warnings.filterwarnings("ignore")

In [3]:
import requests
html_content=requests.get('https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y')
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_content.text, 'html.parser')
table = soup.find_all('table')[1]

data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')  # Find all cells (td) in each row
    cols = [col.text.strip() for col in cols]  # Extract the text and remove leading/trailing whitespace
    data.append(cols)  # Append the row data to the list

df = pd.read_html(str(table))[0]
df.columns = df.iloc[0]  # Set the first row as column names
df = df[1:]  # Remove the first row from the DataFrame
df.reset_index(drop=True, inplace=True)
df['yahoo_ticker'] = df['有價證券代號'] + '.TW'
df.tail(3)

,頁面編號,國際證券編碼,有價證券代號,有價證券名稱,市場別,有價證券別,產業別,公開發行/上市(櫃)/發行日,CFICode,備註,yahoo_ticker
974,975,TW0009946004,9946,三發地產,上市,股票,建材營造業,2013/09/17,ESVUFR,NaN,9946.TW
975,976,TW0009955005,9955,佳龍,上市,股票,綠能環保,2008/01/21,ESVUFR,NaN,9955.TW
976,977,TW0009958009,9958,世紀鋼,上市,股票,鋼鐵工業,2008/03/12,ESVUFR,NaN,9958.TW


In [5]:
df['prices']=prices = [yf.download(df['有價證券代號'].iloc[i]+'.TW', progress=False) for i in range(len(df))]

In [7]:
df

,頁面編號,國際證券編碼,有價證券代號,有價證券名稱,市場別,有價證券別,產業別,公開發行/上市(櫃)/發行日,CFICode,備註,yahoo_ticker,prices
0,1,TW0001101004,1101,台泥,上市,股票,水泥工業,1962/02/09,ESVUFR,NaN,1101.TW,Open High Low ...
1,2,TW0001102002,1102,亞泥,上市,股票,水泥工業,1962/06/08,ESVUFR,NaN,1102.TW,Open High Low ...
2,3,TW0001103000,1103,嘉泥,上市,股票,水泥工業,1969/11/14,ESVUFR,NaN,1103.TW,Open High Low ...
3,4,TW0001104008,1104,環泥,上市,股票,水泥工業,1971/02/01,ESVUFR,NaN,1104.TW,Open High Low ...
4,5,TW0001108009,1108,幸福,上市,股票,水泥工業,1990/06/06,ESVUFR,NaN,1108.TW,Open High Low ...
...,...,...,...,...,...,...,...,...,...,...,...,...
972,973,TW0009944009,9944,新麗,上市,股票,其他業,2002/08/26,ESVUFR,NaN,9944.TW,Open High Low ...
973,974,TW0009945006,9945,潤泰新,上市,股票,其他業,1992/04/30,ESVUFR,NaN,9945.TW,Open High Low ...
974,975,TW0009946004,9946,三發地產,上市,股票,建材營造業,2013/09/17,ESVUFR,NaN,9946.TW,Open High Low ...
975,976,TW0009955005,9955,佳龍,上市,股票,綠能環保,2008/01/21,ESVUFR,NaN,9955.TW,Open High Low ...


In [9]:
otc=pd.read_html\
('https://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&o=htm&d=112/09/20&s=0,desc,1')[0]
otc.columns = [otc.columns[i][1] for i in range(len(otc.columns))]
otc = otc[otc['代號'].str.len() == 4]
otc = otc[['代號','名稱']]
otc = otc[pd.to_numeric(otc['代號'], errors='coerce').notna()]
otc.reset_index(drop=True)
otc['yahoo_ticker'] = otc['代號'] + '.TWO'
otc['prices'] = [yf.download(otc['代號'].iloc[i]+'.TWO', progress=False) for i in range(len(otc))]

In [20]:
otc_prices = otc[['代號','prices']]

In [21]:
otc_prices

,代號,prices
0,9962,Open High Low ...
1,9960,Open High Low ...
2,9951,Open High Low ...
3,9950,Open High Low ...
4,9949,Open High Low ...
...,...,...
10215,1336,Open High Low ...
10216,1268,Open High Low...
10217,1264,Open High Low...
10218,1259,Open High Low...


In [31]:
twse_prices = df[['有價證券代號','prices']]
twse_prices.columns=['代號','prices']

In [40]:
prices_all = pd.concat([twse_prices, otc_prices])

In [43]:
prices_all.to_pickle('prices_all.pkl')